In [2]:
from libhiaac.temporal.datasets.kuhar import BalancedTimeDomainKuHar
from libhiaac.temporal.datasets.dataset import TemporalSample
from libhiaac.temporal.transforms.transforms import Compose, SpectrogramCombiner
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
torch.cuda.is_available()

#val_dataset = BalancedTimeDomainKuHar(dataset_path="data/datasets/KuHar/Balanced", download=True, mode="validation")
#test_dataset = BalancedTimeDomainKuHar(dataset_path="data/datasets/KuHar/Balanced", download=True, mode="test")

True

In [3]:
class MakeSpec:
    def __call__(self, value):
        fig = plt.figure()
        ax = fig.add_axes([0, 0, 1, 1])
        ax.pcolormesh(value[1], value[0], value[2], shading="gouraud", vmin=0, vmax=0.01)
        fig.canvas.draw()
        # Now we can save it to a numpy array.
        data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.close("all")
        return data

spec_combiner = SpectrogramCombiner(fs=100, nperseg=10, nooverlap=1)
make_scpec_fig = MakeSpec()
piller = torchvision.transforms.ToPILImage()
resizer = torchvision.transforms.Resize((32,32))
tensorize = torchvision.transforms.ToTensor()

In [4]:
composer = Compose([spec_combiner, make_scpec_fig, piller, resizer, tensorize])
train_dataset = BalancedTimeDomainKuHar(dataset_path="data/datasets/KuHar/Balanced", download=True, mode="train", transforms=composer)
val_dataset = BalancedTimeDomainKuHar(dataset_path="data/datasets/KuHar/Balanced", download=True, mode="validation", transforms=composer)
test_dataset = BalancedTimeDomainKuHar(dataset_path="data/datasets/KuHar/Balanced", download=True, mode="test", transforms=composer)

'data/datasets/KuHar/Balanced/train.csv' already exists and will not be downloaded again
Done!
'data/datasets/KuHar/Balanced/val.csv' already exists and will not be downloaded again
Done!
'data/datasets/KuHar/Balanced/test.csv' already exists and will not be downloaded again
Done!


In [5]:
data = train_dataset[0]

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/functional.py:136: UserWarning: Legacy tensor constructor is deprecated. Use: torch.tensor(...) for creating tensors from tensor-like objects; or torch.empty(...) for creating an uninitialized tensor with specific sizes. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_new.cpp:478.)
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))


In [6]:
'''DenseNet in PyTorch.'''
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

class Bottleneck(nn.Module):
    def __init__(self, in_planes, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, 4*growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4*growth_rate)
        self.conv2 = nn.Conv2d(4*growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat([out,x], 1)
        return out


class Transition(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_planes)
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.conv(F.relu(self.bn(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
    def __init__(self, block, nblocks, growth_rate=12, reduction=0.5, num_classes=10):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate
        
        num_planes = 2*growth_rate
        self.conv1 = nn.Conv2d(3, num_planes, kernel_size=3, padding=1, bias=False)

        self.dense1 = self._make_dense_layers(block, num_planes, nblocks[0])
        num_planes += nblocks[0]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans1 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense2 = self._make_dense_layers(block, num_planes, nblocks[1])
        num_planes += nblocks[1]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans2 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense3 = self._make_dense_layers(block, num_planes, nblocks[2])
        num_planes += nblocks[2]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans3 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense4 = self._make_dense_layers(block, num_planes, nblocks[3])
        num_planes += nblocks[3]*growth_rate

        self.bn = nn.BatchNorm2d(num_planes)
        self.linear = nn.Linear(num_planes, num_classes)
        self.name = 'DenseNet' + str(sum(nblocks) * 2 + 5)

    def _make_dense_layers(self, block, in_planes, nblock):
        layers = []
        for i in range(nblock):
            layers.append(block(in_planes, self.growth_rate))
            in_planes += self.growth_rate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.trans3(self.dense3(out))
        out = self.dense4(out)
        out = F.avg_pool2d(F.relu(self.bn(out)), 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def DenseNet121():
    return DenseNet(Bottleneck, [6,12,24,16], growth_rate=32, num_classes=17)

def DenseNet169():
    return DenseNet(Bottleneck, [6,12,32,32], growth_rate=32, num_classes=17)

def DenseNet201():
    return DenseNet(Bottleneck, [6,12,48,32], growth_rate=32, num_classes=17)

def DenseNet161():
    return DenseNet(Bottleneck, [6,12,36,24], growth_rate=48, num_classes=17)

def densenet_cifar():
    return DenseNet(Bottleneck, [6,12,24,16], growth_rate=12, num_classes=17)

In [10]:
module = DenseNet121()


In [10]:
def train(train_dataset, model):
    test_loss = 0
    correct = 0
    total = 0
    
    model = model#.to("cuda")
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_dataset):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data.accelerometer
            labels = torch.zeros(17)
            labels[data.meta["class"]] = 1
            
            
            inputs =  torch.unsqueeze(inputs, dim=0)#.to("cuda")
            labels = torch.unsqueeze(labels, dim=0)#.to("cuda")
        
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
                        
            loss = criterion(outputs, torch.max(labels, 1)[1])

            test_loss += loss.item()
            
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            loss.backward()
            optimizer.step()

            acc = correct/total
            # print statistics
            running_loss += loss.item()
            if i % 100 == 0: 
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                print(f"Acc={acc}")
                running_loss = 0.0
        print(f'Epoch acc={acc}')

    print(f'Finished Training: acc={acc}')
    return acc

In [ ]:
torc

In [11]:
model = DenseNet121()
train(train_dataset, model)

[1,     1] loss: 0.030
Acc=0.0
[1,   101] loss: 0.116
Acc=15.524752475247524
[1,   201] loss: 0.684
Acc=11.432835820895523
[1,   301] loss: 1.171
Acc=7.897009966777409
[1,   401] loss: 0.000
Acc=5.927680798004988
[1,   501] loss: 1.489
Acc=4.7445109780439125
[1,   601] loss: 1.679
Acc=3.9550748752079867
[1,   701] loss: 1.739
Acc=3.390870185449358
[1,   801] loss: 0.000
Acc=2.9675405742821472
[1,   901] loss: 1.649
Acc=2.6381798002219754
[1,  1001] loss: 1.648
Acc=2.3746253746253747
[1,  1101] loss: 1.630
Acc=2.1589464123524067
[1,  1201] loss: 0.000
Acc=1.9791840133222314
[1,  1301] loss: 1.563
Acc=1.8270561106840892
[1,  1401] loss: 1.471
Acc=1.6966452533904355


ZeroDivisionError: float division by zero

In [3]:
torch.nn.functional.one_hot(torch.randint(0, 10, (10,)))


tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])